# blah blah

In [1]:

import numpy as np
import pandas as pd

#######################
## Datenaufbereitung ##
#######################


# make imports from pa_lib possible (parent directory of file's directory)
import sys
from pathlib import Path
                                
file_dir = Path.cwd()
parent_dir = file_dir.parent
sys.path.append(str(parent_dir))


from IPython.display import display
pd.options.display.max_columns = None
pd.options.display.max_rows  = None

## Libraries & Settings ##
from pa_lib.file import load_bin
from pa_lib.util import cap_words
from pa_lib.log import time_log, info

import datetime as dt
from dateutil.relativedelta import relativedelta

from pa_lib.data import (
    clean_up_categoricals,
    unfactorize,
)

from pa_lib.data import desc_col

In [4]:
def load_booking_data():
    bd_raw = load_bin("vkprog\\bd_data.feather").rename(
        mapper=lambda name: cap_words(name, sep="_"), axis="columns"
    )
    bd = bd_raw.loc[(bd_raw.Netto > 0)].pipe(clean_up_categoricals)
    return bd

In [5]:
bd_raw = load_booking_data()

2019-10-29 16:44:45 [INFO] Started loading binary file
2019-10-29 16:44:45 [INFO] Reading from file C:\Users\stc\data\vkprog\bd_data.feather
2019-10-29 16:44:45 [INFO] Finished loading binary file in 0.15s (0.77s CPU)


In [47]:
#desc_col(ek_info)

# OMG! I FORGOT ABOUT THE IMPLICIT USE OF COLUMNS IN THE ROW SELECTION!!!

- [x] ``Kleinkunde`` (Über die letzten 4 Jahre nie mehr als 3'000 Umsatz pro Jahr)
- [x] ``Neukunde`` (Alle, die erst im aktuellen Jahr Umsatz hatten)
- [x] ``Insolvenz`` (Kunden mit Bonitäts-Flag "keine Verkäufe" werden markiert)
- [x] ``Umsatz_erreicht`` (80% Netto-Umsatz gem. Vorjahr erreicht) Die Idee war uspruenglich Kunden auszuschliessen, welche schon minimum 80% vom Vorjahresumsatz erreicht haben.
- [x] ``kuerzlich_gebucht`` (in den letzten 2 Monaten erfasste Kampagnen )
- [x] ``kuerzlich_im_aushang`` (Aushangbeginn vor 1 Monat oder später)
- [ ] ``kuerzlich_im_kontakt`` (CRM-Kontakt in den letzten 4 Wochen)
- [ ] ``VB_FILTER_AKTIV`` (in CRM ist eine gültige Sperre für Kunden erfasst) 

In [115]:
ek_info = load_bin("vkprog\\ek_info.feather")

2019-10-30 14:15:59 [INFO] Started loading binary file
2019-10-30 14:15:59 [INFO] Reading from file C:\Users\stc\data\vkprog\ek_info.feather
2019-10-30 14:15:59 [INFO] Finished loading binary file in 0.04s (0.06s CPU)


In [116]:
ek_info.sample(3,random_state=42)

,index,Endkunde_NR,Endkunde,EK_Aktiv,EK_Kam_Betreut,EK_Land,PLZ,GEMEINDE,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Last_Res_Date,First_Res_Year,Last_Res_Year,Last_Aus_Date,EK_HB_Apg_Kurzz,AG_Hauptbetreuer,KANTON,BEZIRK,REGION,GROSSREGION,VERKAUFS_GEBIETS_CODE,VB_VKGEB,Letzter_Kontakt,Kanal,Betreff,last_CRM_Ktkt_date,Letzte_CRM_Ktkts,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Verkaufsberater,Insolvenz
20042,20042,577920,Wizz Air Hungary Kft,1,1,HUNGARY,1185,Budapest,None,WG - Verkehrsbetriebe,410,410,1,2013-06-11,2013,2013,2013-06-17,DEA,None,None,None,None,None,INTERNATIONAL,MAT,CGE,E-Mail,Traffic SR-Actions-2016 - Offre spéciale Power...,2016-09-13,None,NaN,NaN,NaN,NaN,NaN,None,None
10311,10311,503679,Cizen Inc. sports & health,1,0,SCHWEIZ,3011,Bern,None,WG - Handel / Grossverteiler,901,"555,720,800",3,2014-10-01,2008,2014,2015-01-01,BEB,None,BE,Bern-Mittelland,Bern,Region Bern,V-M04,YAM,CJA,E-Mail,Verkauf Bern Mitte 2019 - Begleitkommunikation...,2019-09-24,"LGE,BIM,MRL,ANS,MAL,CJA",NaN,NaN,NaN,NaN,NaN,None,None
13356,13356,520701,Reist Haushaltapparate AG,1,0,SCHWEIZ,4543,Deitingen,Regio Annoncen + Verlag AG,WG - Haushaltsartikel,290,400,1,2009-11-01,2009,2009,2010-02-01,MWY,LOD,SO,Wasseramt,Solothurn,Region Biel–Jura,V-M03,RIN,None,None,None,NaT,None,NaN,NaN,NaN,NaN,NaN,None,None


In [114]:
net_columns = [col for col in ek_info.columns if col.startswith("Net_")] # Net_2015, ..., Net_2019

row_filter = (
    # Insolvenz:
    (ek_info.loc[:,"Insolvenz"] 
         != True
    ) &
    
    # kuerzlich_gebucht (in den letzten 2 Monaten erfasste Kampagnen):
    (ek_info.loc[:,"Last_Res_Date"] 
        < pd.Timestamp.now() - pd.DateOffset(months=2)
    ) &
    
    # kuerzlich_im_aushang (Aushangbeginn vor 1 Monat oder später):
    (ek_info.loc[:,"Last_Aus_Date"] 
        < pd.Timestamp.now() - pd.DateOffset(months=1)
    ) &
    
    # keine Kleinkunden (Ueber die letzten 4 Jahre nie mehr als 3'000 pro Jahr):
    (ek_info.loc[:,net_columns].max(axis=1).fillna(0) 
        > 3000
    ) &
    
    # keine Neukunden (Alle, die erst im aktuellen Jahr Umsatz hatten):
    ((ek_info.loc[:,sorted(net_columns, reverse=True)[1:]]
             .max(axis=1)
             .fillna(0))
        > 0
    ) &
    
    # Umsatz_erreicht (80% Netto-Umsatz gem. Vorjahr erreicht) 
    ((ek_info.loc[:,sorted(net_columns, reverse=True)[0]]
             .fillna(0))
         <= 0.8*(ek_info.loc[:,sorted(net_columns, reverse=True)[1]]
                        .fillna(0))
    )
    
    )

display(ek_info.shape)
display(ek_info.loc[row_filter,:].shape)


(32237, 36)

(6761, 36)

In [94]:
sticcio = sorted(net_columns, reverse=True)[1:]
print(sticcio)

['Net_2018', 'Net_2017', 'Net_2016', 'Net_2015']


In [113]:
ek_info.loc[:,sorted(net_columns, reverse=True)[0]].fillna(0) <= 0.8*ek_info.loc[:,sorted(net_columns, reverse=True)[1]].fillna(0) + 1

0         True
1         True
2         True
3        False
4         True
5        False
6         True
7        False
8         True
9        False
10        True
11       False
12        True
13        True
14       False
15        True
16        True
17       False
18       False
19        True
20        True
21       False
22        True
23        True
24        True
25       False
26        True
27       False
28        True
29        True
30        True
31        True
32       False
33       False
34        True
35       False
36        True
37        True
38        True
39       False
40        True
41        True
42       False
43        True
44       False
45       False
46        True
47        True
48        True
49        True
50       False
51        True
52       False
53       False
54        True
55        True
56       False
57        True
58        True
59       False
60       False
61       False
62        True
63        True
64        True
65        True
66        

In [107]:
sorted(net_columns, reverse=True)[1]

'Net_2018'

In [95]:
print(net_columns)

['Net_2015', 'Net_2016', 'Net_2017', 'Net_2018', 'Net_2019']


In [98]:
#ek_info.loc[:,net_columns].max(axis=1).fillna(0) > 3000

In [69]:
#ek_info.loc[:,"Last_Res_Date"] > pd.Timestamp.now() - pd.DateOffset(months=2)

In [6]:
bd_raw.sample(5, random_state=42)

,Endkunde_NR,Endkunde,EK_Abc,EK_Boni,EK_Plz,EK_Ort,EK_Land,EK_HB_Apg_Kurzz,EK_Kam_Betreut,EK_Aktiv,Agentur,AG_Hauptbetreuer,Verkaufsberater,Endkunde_Branchengruppe_ID,Endkunde_Branchengruppe,Endkunde_Nbranchengruppe_ID,Endkunde_Nbranchengruppe,Endkunde_Branchenkat_ID,Endkunde_Branchenkat,Endkunde_Nbranchenkat_ID,Endkunde_Nbranchenkat,Auftrag_Branchengruppe_ID,Auftrag_Branchengruppe,Auftrag_Nbranchengruppe_ID,Auftrag_Nbranchengruppe,Auftrag_Branchenkat_ID,Auftrag_Branchenkat,Auftrag_Nbranchenkat_ID,Auftrag_Nbranchenkat,Agps_NR,Segment,KV_NR,KV_Typ,Kampagnen_Status,Kampagne_Erfassungsdatum,Kampagne_Beginn,Auftragsart,Res_Dat,Annullation_Datum,Aush_Von,Dauer,Vertrag,Brutto,Netto,Agglo,PF,Kamp_Beginn_Jahr,Kamp_Beginn_KW,Kamp_Beginn_KW_2,Kamp_Beginn_KW_4,Kamp_Erfass_Jahr,Kamp_Erfass_KW,Kamp_Erfass_KW_2,Kamp_Erfass_KW_4
1017991,592834,Carta turistica SA,E,gut,6901,Lugano,SCHWEIZ,SOL,0,1,NaN,NaN,ROS,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,405,WG - Reisen / Hotels / Sanatorien,NaN,NaN,10,WB - Freizeit / Touristik,NaN,NaN,2050838,APG|SGA,811509,KPG,4,2015-03-10,2015-06-15,Kommerziell,2015-03-27,NaT,2015-06-15,14,Nein,3733,3733,"A5192,A5250",F12,2015,25,25,25,2015,11,11,9
1197937,168605,C&A Mode AG,A,gut,6341,Baar,SCHWEIZ,DEA,1,1,"R. Koller & L. Della Piana Koller, Consulting",KUR,KUR,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,195,WG - Bekleidung / Wäsche,901,WG - Handel / Grossverteiler,04,WB - Bekleidung / persönlicher Bedarf,14,WB - Handel,1813376,APG|SGA,754283,KPG,4,2013-11-26,2014-05-05,Kommerziell,2013-11-26,NaT,2014-05-05,14,Nein,2428,1991,"90306,90329,90404,91103,91509,92583,93293,9378...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2014,19,19,17,2013,48,47,45
1137867,536344,Küry Automobile AG,D,gut,8803,Rüschlikon,SCHWEIZ,MOE,0,1,X-TRA RAINER TRUTTMANN,BGI,DSC,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,400,WG - Automarkt,NaN,NaN,13,WB - Verkehr,NaN,NaN,1768257,Traffic,742309,KDVTLG,4,2013-09-01,2013-12-01,Traffic Auftrag,2013-09-01,NaT,2013-12-01,365,Ja,7500,7500,A0261,"F12,F24,Ganzheck",2013,48,47,45,2013,35,35,33
865735,104912,APG|SGA Rail,A,gut,8027,Zürich,SCHWEIZ,YAM,0,1,NaN,NaN,JZE,720,WG - Dienstleistung,NaN,NaN,15,WB - Dienstleistung,NaN,NaN,980,WG - Spitäler / Kliniken / Kurzentren / Aerzte,NaN,NaN,21,WB - Spitäler/ Kliniken/ Kurzentren/ Aerzte,NaN,NaN,1672038,Rail,717049,KPG,4,2013-01-09,2013-01-28,Aushangauftrag Partner,2013-01-09,NaT,2013-01-28,7,Nein,576,576,"90306,90329,90404,90412,90768,91103,91362,9137...","F12,F12L,F200,F200L,F24,F4,F4C,F4S,NeonSign,Pr...",2013,5,5,5,2013,2,1,1
165668,144291,AllBlues Konzert AG,D,gut,8404,Winterthur,SCHWEIZ,ACO,0,1,NaN,NaN,ACO,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,990,WG - Veranstaltungen,NaN,NaN,22,WB - Veranstaltungen,NaN,NaN,2798438,Digital Sales,968021,KPG,4,2018-10-04,2018-10-15,Kommerziell,2018-10-04,NaT,2018-10-29,1,Nein,168,168,"A0230,A0261,A0351,A1061,A2701,A3203,A4001,A558...","City eBoard,City ePanel,Escalator ePanel,F12,F...",2018,42,41,41,2018,40,39,37


In [8]:
set(bd_raw.loc[:,"EK_Boni"])

{'gegen Vorauszahlung', 'gut', 'keine Verkäufe', nan}

In [45]:
def insolvenz(bd_raw):
    row_select = (bd_raw.loc[:,"EK_Boni"] == "keine Verkäufe")
    insolvenz  = bd_raw.loc[row_select,["Endkunde_NR", "EK_Boni"]].drop_duplicates()
    insolvenz.loc[:,"Insolvenz"] = True
    return insolvenz.loc[:,["Endkunde_NR", "Insolvenz"]]

In [46]:
insolvenz(bd_raw=bd_raw)

,Endkunde_NR,Insolvenz
131,505733,True
999,107064,True
1125,555216,True
1355,517905,True
1543,549402,True
1692,129530,True
2494,534398,True
2739,589390,True
2914,520299,True
3320,600785,True


In [32]:
desc_col(insolvenz)

,DTYPE,NULLS,UNIQUE
Endkunde_NR,int64,0/613,613
EK_Boni,category,0/613,1
Insolvenz,bool,0/613,1


In [16]:
ek_info.sample(5,random_state=42)

,index,Endkunde_NR,Endkunde,EK_Aktiv,EK_Kam_Betreut,EK_Land,PLZ,GEMEINDE,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Last_Res_Date,First_Res_Year,Last_Res_Year,Last_Aus_Date,EK_HB_Apg_Kurzz,AG_Hauptbetreuer,KANTON,BEZIRK,REGION,GROSSREGION,VERKAUFS_GEBIETS_CODE,VB_VKGEB,Letzter_Kontakt,Kanal,Betreff,Letzte_CRM_Ktkts,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Verkaufsberater
20042,20042,577920,Wizz Air Hungary Kft,1,1,HUNGARY,1185,Budapest,None,WG - Verkehrsbetriebe,410,410,1,2013-06-11,2013,2013,2013-06-17,DEA,None,None,None,None,None,INTERNATIONAL,MAT,CGE,E-Mail,Traffic SR-Actions-2016 - Offre spéciale Power...,None,NaN,NaN,NaN,NaN,NaN,None
10311,10311,503679,Cizen Inc. sports & health,1,0,SCHWEIZ,3011,Bern,None,WG - Handel / Grossverteiler,901,"555,720,800",3,2014-10-01,2008,2014,2015-01-01,BEB,None,BE,Bern-Mittelland,Bern,Region Bern,V-M04,YAM,CJA,E-Mail,Verkauf Bern Mitte 2019 - Begleitkommunikation...,"LGE,BIM,MRL,ANS,MAL,CJA",NaN,NaN,NaN,NaN,NaN,None
13356,13356,520701,Reist Haushaltapparate AG,1,0,SCHWEIZ,4543,Deitingen,Regio Annoncen + Verlag AG,WG - Haushaltsartikel,290,400,1,2009-11-01,2009,2009,2010-02-01,MWY,LOD,SO,Wasseramt,Solothurn,Region Biel–Jura,V-M03,RIN,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None
17149,17149,555869,Wülser Lostorf AG,1,0,SCHWEIZ,4654,Lostorf,None,WG - Dienstleistung,720,"295,720",2,2019-10-01,2011,2019,2020-01-01,MWY,None,SO,Gösgen,Aarau–Olten,Aareland,V-M03,RIN,CFU,Besuch,Umplatzierung BOGG da Ausrangierung,CFU,0.0,0.0,0.0,4000.0,4000.0,"CFU,MWY"
10714,10714,506552,Christoph Bugnon,0,0,SCHWEIZ,6253,Uffikon,None,WG - Veranstaltungen,990,990,1,2009-05-08,2009,2009,2009-06-08,BNI,None,LU,Willisau,Sursee,Zentralschweiz,V-M07,JCO,None,None,None,None,NaN,NaN,NaN,NaN,NaN,None


In [18]:
ek_info.sort_values("Last_Res_Date", ascending=False).head(5)

,index,Endkunde_NR,Endkunde,EK_Aktiv,EK_Kam_Betreut,EK_Land,PLZ,GEMEINDE,Agentur,EK_BG,EK_BG_ID,Auftrag_BG_ID,Auftrag_BG_Anz,Last_Res_Date,First_Res_Year,Last_Res_Year,Last_Aus_Date,EK_HB_Apg_Kurzz,AG_Hauptbetreuer,KANTON,BEZIRK,REGION,GROSSREGION,VERKAUFS_GEBIETS_CODE,VB_VKGEB,Letzter_Kontakt,Kanal,Betreff,Letzte_CRM_Ktkts,Net_2015,Net_2016,Net_2017,Net_2018,Net_2019,Verkaufsberater
26569,26569,621738,AGLR lebendiges Reinach,1,0,SCHWEIZ,4153,Reinach BL,None,WG - Veranstaltungen,990,"990,720",2,2019-10-20,2016,2019,2019-12-02,RPO,None,BL,Arlesheim,Reinach (BL)–Allschwil,Region Basel,V-M01,RPO,CKT,E-Mail,Mitte - Aktionen 2019 - ROF | 50% City ePanels...,"ROF,SGU1,MAL,MDR,BAP,GPE,CKT,ACH",0.0,324.0,783.0,0.0,521.0,"RPO,ROF"
28317,28317,633314,IG Mall of Switzerland,1,0,SCHWEIZ,6030,Ebikon,ACE 2 ACE outdoor media ag,WG - Handel / Grossverteiler,901,"901,400",2,2019-10-18,2017,2019,2020-03-09,JCO,NYI,LU,Luzern-Land,Luzern,Zentralschweiz,V-M07,JCO,HUN,E-Mail,K1016440 | Offerte: Buswerbeaushänge IG Mall o...,"EGL,BBO,JCO,MDR,BNI,NAZ,BIM,BAP,JEN,GPE,MAL,HUN",0.0,0.0,115366.0,213253.0,86155.0,"NYI,JCO,LAH,BAM,ZHU,MFI,CGA,BBO"
10418,10418,504417,Ensemble symphonique de Neuchâtel ESN,1,0,SCHWEIZ,2000,Neuchâtel,None,WG - Veranstaltungen,990,990,1,2019-10-18,2008,2019,2020-05-04,VIT,None,NE,Cant. de Neuchâtel,Neuchâtel,Region Neuenburg,V-W04,VIT,VJE,E-Mail,ESN - Campagnes n° 1016312 & 1016313,"SFO,VTO,CGE,KRU,BAP,LIC,MFB,WSA,VIT,MTO,OHA,VJE",5216.0,12658.0,10315.0,6060.0,5509.0,"VIT,VTO,MFB,MTO"
323,323,103092,CERTINA SA,1,1,SCHWEIZ,2503,Biel/Bienne,Outdoor Media AG,WG - Persönlicher Bedarf,555,"555,400",2,2019-10-18,2007,2019,2020-02-24,ROE,NYI,BE,Biel/Bienne,Biel/Bienne,Region Biel–Jura,V-M03,RIN,ROE,E-Mail,AW: Neujahrspackages KW 01-06/ 2020,"ROE,VTO,FIP,LAH,BBO,KRU,HUN,SOL,SDU,CJA,CGE,CF...",263089.0,243488.0,216860.0,280961.0,263447.0,"LAH,NYI,ROE,MTO,BAM,CFU"
1961,1961,119916,Théâtre du Loup,1,0,SCHWEIZ,1227,Les Acacias,None,WG - Veranstaltungen,990,990,1,2019-10-18,2008,2019,2020-05-25,SIB,None,GE,Cant. de Genève,Vernier–Lancy,Region Genf,V-W01,SIB,AAN,E-Mail,Théâtre du Loup - Offres 1er semestre 20,"HOF,BAP,LIC,AAN,SIB",7245.0,4200.0,2121.0,1512.0,3302.0,"HOF,SIB"
